# Creating new asset classes

This notebook shows how to create instrument with custom fields, also known as properties

# Setup LUSID

In [1]:
# Import system packages

import os

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.api as la
import lusid.models as lm
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
    format_transactions_response
)

# Import data wrangling and data management packages
import pandas as pd
import numpy as np
import json
import pytz
from datetime import datetime

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

# Define scope

In [2]:
scope = "poc-sample"

# Define custom properties for new assets

In [3]:
properties_api = api_factory.build(la.PropertyDefinitionsApi)
results = []

instrument_properties = [
    ("StrikePrice", "Strike Price", "number"),
    ("Maturity", "Maturity", "date"),
    ("FloatingRateOption", "Floating Rate Option", "string"),
    ("Notional", "Notional", "number"),
    ("StartDate", "Start Date", "date")
]

for code, description, data_type in instrument_properties:

    try:
        properties_api.create_property_definition(
            lm.CreatePropertyDefinitionRequest(
                domain="Instrument",
                scope=scope,
                code=code,
                display_name=description,
                life_time="Perpetual",
                value_required=False,
                data_type_id=lm.resource_id.ResourceId(scope="system", code=data_type)
            )
        )
        results.append({"Property": code, "success": True})
    except ApiException as e:
        results.append({"Property": code,"success": False, "error": json.loads(e.body)["name"], "Error Message": json.loads(e.body)["title"]})
        if json.loads(e.body)["name"] != "PropertyAlreadyExists":
            raise
            

pd.DataFrame(data=results)

,Property,success,error,Error Message
0,StrikePrice,False,PropertyAlreadyExists,Error creating Property Definition 'Instrument...
1,Maturity,False,PropertyAlreadyExists,Error creating Property Definition 'Instrument...
2,FloatingRateOption,False,PropertyAlreadyExists,Error creating Property Definition 'Instrument...
3,Notional,False,PropertyAlreadyExists,Error creating Property Definition 'Instrument...
4,StartDate,False,PropertyAlreadyExists,Error creating Property Definition 'Instrument...


# Load in the source data

In [4]:
df = pd.read_excel("./exotics-properties.xlsx")    
df

,Id,ClientInternal,Instrument Name,Strike Price,Maturity,Floating Rate Option,Notional,Start Date
0,SDB368,BS-1,GS - Barrier Swap,4.5,2025-06-01,EUR-EURIBOR-REUTERS,5000000,2020-06-01


# Load the instruments

 Create the assets and additionally add Instrument properties so they can be viewed in holdings

In [5]:
results = []
instruments_request = {
    row["Id"]: lm.LusidInstrumentDefinition(
        name=row["Instrument Name"],
        identifiers={"ClientInternal": lm.InstrumentIdValue(row["ClientInternal"])},
        properties=[
            lm.ModelProperty(key=f"Instrument/{scope}/StrikePrice",
                             value=lm.PropertyValue(metric_value=lm.MetricValue(value=row["Strike Price"]))),
            lm.ModelProperty(key=f"Instrument/{scope}/Maturity",
                             value=lm.PropertyValue(label_value=row["Maturity"])),
            lm.ModelProperty(key=f"Instrument/{scope}/FloatingRateOption",
                             value=lm.PropertyValue(label_value=row["Floating Rate Option"])),
            lm.ModelProperty(key=f"Instrument/{scope}/Notional",
                             value=lm.PropertyValue(metric_value=lm.MetricValue(value=row["Notional"]))),
            lm.ModelProperty(key=f"Instrument/{scope}/StartDate",
                             value=lm.PropertyValue(label_value=row["Start Date"])),
        ]
    )
    
    for _, row in df.iterrows()
}

for prop in instruments_request["SDB368"].properties:
    results.append({"Property Definition": prop.key, "Creation Success": True })
pd.DataFrame(data=results)

,Property Definition,Creation Success
0,Instrument/poc-sample/StrikePrice,True
1,Instrument/poc-sample/Maturity,True
2,Instrument/poc-sample/FloatingRateOption,True
3,Instrument/poc-sample/Notional,True
4,Instrument/poc-sample/StartDate,True


Create the instruments

In [6]:
instruments_api = api_factory.build(la.InstrumentsApi)
result = instruments_api.upsert_lusid_instruments(request_body=instruments_request)

pd.DataFrame(data=[{"Success": True if len(result.failed) < 1 else False, "Failed": len(result.failed)}])

,Success,Failed
0,True,0


#  Create the portfolio

In [7]:
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

# Create portfolio with properties
subholding_key = {}
created_date = "2010-01-01T00:00:00.000000+00:00"

# create request body
portfolio_request = lm.CreateTransactionPortfolioRequest(
    display_name="Exotics",
    code="exotics",
    base_currency="GBP",
    created=created_date
)

# Upload new portfolio to LUSID
results = []
try:
    response = transaction_portfolios_api.create_portfolio(
        scope=scope, create_transaction_portfolio_request=portfolio_request
    )
    results.append({"success": True, "Message": f"portfolio '{response.id.code}', in scope {scope} created"})

except ApiException as e:
    results.append({"success": False, "error": json.loads(e.body)['name'], "Error Message": json.loads(e.body)['title']})
    if json.loads(e.body)['name'] != "PortfolioWithIdAlreadyExists":
        raise

pd.DataFrame(data=results)

,success,error,Error Message
0,False,PortfolioWithIdAlreadyExists,Could not create a portfolio with id exotics b...


# Define properties for transactions

In [8]:
df_transactions = pd.read_excel("./exotics-transaction.xlsx")
df_transactions

results = []
try:
    # Call LUSID to create a property
    property_response = properties_api.create_property_definition(
        create_property_definition_request=lm.CreatePropertyDefinitionRequest(
            domain="Transaction",
            scope=scope,
            code="FeeAmount",
            value_required=False,
            display_name="Fee Amount",
            data_type_id=lm.ResourceId(scope="system", code="number"),
        )
    )
    results.append({"success": True, "Message": "Property created"})
except ApiException as e:
    results.append({"success": False, "error": json.loads(e.body)['name'], "Error Message": json.loads(e.body)['title']})
    if json.loads(e.body)['name'] != "PropertyAlreadyExists":
        raise

pd.DataFrame(data=results)

,success,error,Error Message
0,False,PropertyAlreadyExists,Error creating Property Definition 'Transactio...


# Configure Transaction Types

In [9]:
# First create a list of custom sides which will be used on the new transaction type

system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)

side_list = [
    lm.SideConfigurationDataRequest(
            side="Fee",
            security="Txn:SettleCcy",
            currency="Txn:TradeCurrency",
            rate="SettledToPortfolioRate",
            units=f"Transaction/{scope}/FeeAmount",
            amount=f"Transaction/{scope}/FeeAmount"
        )
    ]


for side in side_list:
    
    current_sides = [side.side for side in system_configuration_api.list_configuration_transaction_types().side_definitions]
    
    if side.side in list(current_sides):
        
        print(f"{side.side} already exists in LUSID")
    
    else:
        
        response = system_configuration_api.create_side_definition(side_configuration_data_request = side)
        
        print(f"{side.side} has been created in LUSID")

Fee has been created in LUSID


In [10]:
# Create the new transaction types using the new sides

try:

    system_configuration_api.create_configuration_transaction_type(
        transaction_configuration_data_request=lm.TransactionConfigurationDataRequest(
            aliases=[
                lm.TransactionConfigurationTypeAlias(
                    type="otc",
                    description="A purchase transaction from System X",
                    transaction_class="Basic",
                    transaction_group="default",
                    transaction_roles="LongLonger",
                )
            ],
            movements=[
                lm.TransactionConfigurationMovementData(
                    movement_types="StockMovement",
                    side="Side1",
                    direction=1,
                    properties={},
                    mappings=[],
                ),
                lm.TransactionConfigurationMovementData(
                    movement_types="CashCommitment",
                    side="Fee",
                    direction=-1,
                    properties={},
                    mappings=[],
                )
            ]
        )
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

# Create transactions for the new asset

In [11]:
# Upsert transactions
transactions_request = []
txn_response = []
created_date = "2010-01-01T00:00:00.000000+00:00"
for row, txn in df_transactions.iterrows():
    # build request body
    transactions_request.append(
        lm.TransactionRequest(
            transaction_id=txn["Transaction ID"],
            type="otc",
            instrument_identifiers={ f"Instrument/default/ClientInternal": txn["ClientInternal"] },
            transaction_date=created_date,
            settlement_date=created_date,
            units=txn["Units"],
            transaction_currency=txn["Transaction Currency"],
            total_consideration=lm.CurrencyAndAmount(
                amount=txn["Total Consideration"], currency=txn["Transaction Currency"]
            ),
            counterparty_id=txn["Counterparty"],
            properties={
                f"Transaction/{scope}/FeeAmount": lm.PerpetualProperty(
                    key=f"Transaction/{scope}/FeeAmount",
                    value=lm.PropertyValue(
                        metric_value=lm.MetricValue(
                            value=txn["Fee Amount"])),
                )
            },
        )
    )

    # Make Upsert Transactions call to LUSID
    txn_response.append(
        transaction_portfolios_api.upsert_transactions(
            scope=scope, code="exotics", transaction_request=transactions_request
        )
    )

pd.DataFrame(data=[{"Transactions upserted": len(txn_response)}])

,Transactions upserted
0,1
